In [1]:
import numpy as np
import astropy.io.fits as pyfits

In [2]:
#### User input ####

## path to the data of interest
region_name = "RCW79"
data_path = "../data_CIIcorr/data_FEEDBACK/{region}/".format(region = region_name)

## first 5: Herschel, then 5: Spitzer, last: upGREAT file
#name_list = ["Herschel_PACS_160_36_15", 
#             "Herschel_PACS_70_36_15",
#             "Herschel_SPIRE_250_36_15",
#             "Herschel_SPIRE_350_36_15",
#             "Herschel_SPIRE_500_36_15",
#             "Spitzer_IRAC_3p6_36_15",
#             "Spitzer_IRAC_4p5_36_15",
#             "Spitzer_IRAC_5p8_36_15",
#             "Spitzer_IRAC_8_36_15",
#             "upGREAT_CII_36_15_integrated"
#            ]

## for the 100 micron data
name_list = ["Herschel_PACS_100_36_15"]

In [3]:
def check_edge(row, col, data, val, edge = 3):
    ## First: check if it's near the edge of the map
    cond1 = (row <= edge) or (row >= len(data)-edge) or (col <= edge) or (col >= len(data[0])-edge)
    if cond1:
        return np.nan
    
    ## Check if no nan value exists nearby
    for x in range(1, edge+1):
        for y in range(1, edge+1):
            cond2 = (np.isnan(data[row-x][col-y]) or np.isnan(data[row-x][col+y])
                    or np.isnan(data[row+x][col-y]) or np.isnan(data[row+x][col+y]))
            if cond2:
                return np.nan
    
    return val

In [4]:
## loop over all files
for name in name_list:
    print("{reg}_{f}.fits".format(reg = region_name, f = name))
    ## get the data
    hdu = pyfits.open("{p}{reg}_{f}.fits".format(p = data_path, reg = region_name, f = name))
    data = hdu[0].data
    
    ## create the new output data
    new_data = np.copy(data)
    
    ## run through the data
    for row in range(0, len(data)):
        for col in range(0, len(data[0])):
            if(~np.isnan(data[row][col])):
                new_data[row][col] = check_edge(row, col, data, data[row][col])
    
    ## save the new fits file
    new_hdu = pyfits.PrimaryHDU(new_data, hdu[0].header)
    new_hdu.writeto("{p}{reg}_{f}_noEdge.fits".format(p = data_path, reg = region_name, f = name), overwrite = True)

RCW79_Herschel_PACS_100_36_15.fits
